# 自作のDDPGの推論用コード

In [1]:
import os
import sys
import time
import glob
import copy
import logging
from dataclasses import dataclass, asdict, is_dataclass
from types import SimpleNamespace

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gymnasium as gym

from myActivator import tanhAndScale

from myAgent import DDPGAgent


# -----------------------------
# 3) 非ブロッキング key 入力（q で終了）
#   - Windows: msvcrt
#   - Unix端末: termios + select
#   - Jupyter等: 無効（Ctrl+C で停止）
# -----------------------------
def make_nonblocking_key_reader():
    restore_fn = None

    # Windows
    if os.name == "nt":
        import msvcrt

        def read_key():
            if msvcrt.kbhit():
                return msvcrt.getwch()
            return None

        return read_key, restore_fn

    # Unix系（端末）
    if sys.stdin.isatty():
        import termios
        import tty
        import select

        fd = sys.stdin.fileno()
        old = termios.tcgetattr(fd)
        tty.setcbreak(fd)

        def restore():
            termios.tcsetattr(fd, termios.TCSADRAIN, old)

        restore_fn = restore

        def read_key():
            if select.select([sys.stdin], [], [], 0.0)[0]:
                return sys.stdin.read(1)
            return None

        return read_key, restore_fn

    # Jupyter 等
    def read_key():
        return None

    return read_key, restore_fn


# -----------------------------
# 4) モデルファイル選択（最新を自動で拾う）
# -----------------------------
MODEL_GLOB = "./models/ddpg_final_*.pth"
candidates = sorted(glob.glob(MODEL_GLOB))
if len(candidates) == 0:
    raise FileNotFoundError(f"No checkpoint found: {MODEL_GLOB}")
MODEL_PATH = candidates[-1]
print("Using checkpoint:", MODEL_PATH)


# -----------------------------
# 5) checkpoint から Config を復元して agent を作る
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 先に ckpt を読み、config dict を取り出す
ckpt_for_cfg = torch.load(MODEL_PATH, map_location=device, weights_only=False)
cfg_dict = ckpt_for_cfg["config"]  # save_all() で保存した config

# dict -> 属性アクセスできる形にする（SimpleNamespace）
# 例: cfg.Q_net_sizes, cfg.u_ulim, ...
cfg = SimpleNamespace(**cfg_dict)

agent = DDPGAgent(Config=cfg, device=device)
agent.load_all(MODEL_PATH, map_location=device)

print("cuda available:", torch.cuda.is_available())
print("agent device:", agent.device)
print("P_net device:", next(agent.P_net.parameters()).device)


# -----------------------------
# 6) 環境を作って推論ループ（描画あり）
# -----------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    stream=sys.stdout,
    datefmt="%H:%M:%S",
)

env = gym.make("Pendulum-v1", render_mode="human")

read_key, restore_key_state = make_nonblocking_key_reader()

try:
    episode = 0
    while True:
        obs, info = env.reset()
        terminated = False
        truncated = False
        ep_return = 0.0
        ep_steps = 0
        episode += 1

        while True:
            # q で停止（端末でのみ有効な場合がある）
            ch = read_key()
            if ch is not None and str(ch).lower() == "q":
                print("\nQuit requested by keypress.")
                raise KeyboardInterrupt

            # 推論（ノイズ無し）
            action = agent.step(obs)

            # 1 step
            obs, reward, terminated, truncated, info = env.step(action)
            env.render()

            ep_return += float(reward)
            ep_steps += 1

            if terminated or truncated:
                print(f"Episode {episode:4d} | return = {ep_return: .3f} | steps = {ep_steps}")
                break

            # 見やすさのため（任意）
            # time.sleep(1.0 / 60.0)

except KeyboardInterrupt:
    pass

finally:
    if restore_key_state is not None:
        try:
            restore_key_state()
        except Exception:
            pass
    env.close()
    print("Evaluation finished.")


Using checkpoint: ./models/ddpg_final_20260131_141220.pth
cuda available: True
agent device: cuda
P_net device: cuda:0
Evaluation finished.
